### Import packages and Gaia tables

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table, join
from astropy.coordinates import SkyCoord
from astropy import units as u

from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
#tables = Gaia.load_tables(only_names=True)
#for table in tables:
#    print(table.name)

### Load Gaia_source data

In [2]:
meta_source = Gaia.load_table('gaiadr3.gaia_source')
meta_source
print(meta_source)

meta_galaxies = Gaia.load_table('gaiadr3.galaxy_candidates')
print(meta_galaxies)

#meta_astrophys = Gaia.load.table('gaiadr3.astrophysical_parameters_supp')
#meta_astrophys
#print(meta_astrophys)

TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152
TAP Table name: gaiadr3.galaxy_candidates
Description: This table contains parameters derived from various modules dedicated to the classification and characterisation of sources considered as galaxy candidates. This table has been constructed with the intention to be complete rather than pure and, as such,

### List gaia source data

# Search Gaia DR3 host galaxy

In [3]:
id = "ZTF25acahpls"
ra = 272.464618
dec = -20.987513
radius_arcsec = 5


In [4]:

query_gaia = f"""
SELECT
  DISTANCE(POINT('ICRS', {ra}, {dec}), POINT('ICRS', s.ra, s.dec)) AS ang_sep,
  s.source_id,
  s.ra, s.dec,
  s.ref_epoch,
  s.classprob_dsc_combmod_galaxy,
  gc.source_id AS galaxy_candidate_source_id
FROM gaiadr3.gaia_source AS s
LEFT JOIN gaiadr3.galaxy_candidates AS gc
  ON gc.source_id = s.source_id
WHERE 1 = CONTAINS(
  POINT('ICRS', s.ra, s.dec),
  CIRCLE('ICRS', {ra}, {dec}, {radius_arcsec}/3600.)
)
ORDER BY ang_sep ASC
"""


In [5]:
gaia_job = Gaia.launch_job(query_gaia)
results_gaia = gaia_job.get_results()
results_gaia

ang_sep,source_id,ra,dec,ref_epoch,classprob_dsc_combmod_galaxy,galaxy_candidate_source_id
,,deg,deg,yr,,
float64,int64,float64,float64,float64,float32,int64
0.00021075697712312488,4094089236026720512,272.464788295336,-20.987374659063054,2016.0,5.0959643e-13,--
0.0009020367194338521,4094089240340869760,272.4638476815814,-20.98805742892604,2016.0,3.314698e-09,--
0.000915317700872032,4094089240323849216,272.46477900362373,-20.988415889697198,2016.0,--,--
0.0013254511998529825,4094089240323849472,272.46565301665777,-20.988420188051194,2016.0,3.224387e-08,--


In [6]:
# Convert Astropy Table to Pandas DataFrame
gaia_df = results_gaia.to_pandas()

# Convert ang_sep from degrees to arcseconds
gaia_df['ang_sep_arcsec'] = gaia_df['ang_sep'] * 3600.0

# Reorder and rename columns
gaia_df = gaia_df[['source_id', 'ang_sep_arcsec', 'ra', 'dec', 
         'classprob_dsc_combmod_galaxy', 'galaxy_candidate_source_id']]

# Display with limited precision for readability
pd.set_option('display.precision', 3)
display(id)
display(gaia_df)


'ZTF25acahpls'

,source_id,ang_sep_arcsec,ra,dec,classprob_dsc_combmod_galaxy,galaxy_candidate_source_id
0,4094089236026720512,0.759,272.465,-20.987,5.096e-13,<NA>
1,4094089240340869760,3.247,272.464,-20.988,3.315e-09,<NA>
2,4094089240323849216,3.295,272.465,-20.988,NaN,<NA>
3,4094089240323849472,4.772,272.466,-20.988,3.224e-08,<NA>


# Cross Match with 2MASS Extended Source Database
This database includes 1.6M galaxies, with some number of other extended sources.

In [7]:
# This code searches the 2MASS point source catalog. This data is not useful for identifying host galaxies.

# Example input coordinates (RA, Dec in degrees)
input_coords = SkyCoord([ra], [dec], unit=(u.deg, u.deg), frame='icrs')

# Set Vizier to return 2MASS XSC (catalog II/246) with a search radius
Vizier.columns = ['2MASS', 'RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag', 'Qflg', 'Ndet', 'Xflg', 'Aflg', 'opt']
Vizier.ROW_LIMIT = -1  # remove row limit
radius = 5 * u.arcsec

# Query 2MASS XSC around each input coordinate
results = []
for coord in input_coords:
    result = Vizier.query_region(coord, radius=radius, catalog='II/246')
    if result:
        df = result[0].to_pandas()
        df['input_RA'] = coord.ra.deg
        df['input_Dec'] = coord.dec.deg
        results.append(df)

# Combine results into a single DataFrame
if results:
    matched_df = pd.concat(results, ignore_index=True)
    print(matched_df[['2MASS', 'RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag', 'Qflg', 'Ndet', 'Xflg', 'Aflg', 'opt']])
else:
    print("No matches found.")

              2MASS  RAJ2000  DEJ2000  Jmag   Hmag   Kmag Qflg    Ndet  Xflg  \
0  18095155-2059144  272.465  -20.987  6.44  5.094  4.366  AAE  663626     0   

   Aflg opt  
0     0   U  


In [8]:
import pyvo  #Virtual Observatory (VO) services. Supports TAP and ADQL.

# Connect to Vizier's TAP service
tap_service = pyvo.dal.TAPService("https://tapvizier.u-strasbg.fr/TAPVizieR/tap")

# List available tables (optional sanity check)
tables = tap_service.tables
for name in tables.keys():
    if "VII/233" in name:
        print(name)

"VII/233/xsc"


In [9]:
# Query the 2MASS Extended Source Catalog (XSC, VII/233) with quality cuts.
# Define your coordinate variables
ra = ra
dec = dec
radius_arcsec = radius_arcsec

# Format the ADQL query using an f-string
adql_query = f"""
SELECT *
FROM "VII/233/xsc"
WHERE 1=CONTAINS(
  -- Select sources within a circular region around your target coordinate
  POINT('ICRS', "RAJ2000", "DEJ2000"),
  CIRCLE('ICRS', {ra}, {dec}, {radius_arcsec / 3600.0})
)

-- Photometric cut: include fainter galaxies down to Ks = 16
AND "K.ext" < 16

-- Morphology cut: require semi-major axis > 5 arcsec (well-resolved sources)
AND "r3sig" > 5

-- Shape cut: axis ratio > 0.5 (exclude extremely elongated or edge-on sources)
AND "Sb/a" > 0.5

-- Photometric uncertainty cut: Ks-band uncertainty < 0.1 mag
AND "e_K.ext" < 0.1

-- Reliability cut: Spa < 2 (lower values indicate better positional reliability)
AND "Spa" < 2
"""

# Run the query
job = tap_service.search(adql_query)
results = job.to_table()
results.pprint(max_width=120)

recno JDobs 2MASX RAJ2000 DEJ2000 supRAdeg supDEdeg dens ... HbgPer HbgAmp KbgPer KbgAmp Jtrack Htrack Ktrack extKey
        d           deg     deg     deg      deg         ... arcsec        arcsec                                   
----- ----- ----- ------- ------- -------- -------- ---- ... ------ ------ ------ ------ ------ ------ ------ ------


In [10]:
# Query the 2MASS Extended Source Catalog (XSC, VII/233) without quality cuts.
# Define your coordinate variables
ra = ra
dec = dec
radius_arcsec = radius_arcsec

# Format the ADQL query using an f-string
adql_query = f"""
SELECT *
FROM "VII/233/xsc"
WHERE 1=CONTAINS(
  -- Select sources within a circular region around your target coordinate
  POINT('ICRS', "RAJ2000", "DEJ2000"),
  CIRCLE('ICRS', {ra}, {dec}, {radius_arcsec / 3600.0})
)
"""

# Run the query
job = tap_service.search(adql_query)
results = job.to_table()
results.pprint(max_width=120)

recno JDobs 2MASX RAJ2000 DEJ2000 supRAdeg supDEdeg dens ... HbgPer HbgAmp KbgPer KbgAmp Jtrack Htrack Ktrack extKey
        d           deg     deg     deg      deg         ... arcsec        arcsec                                   
----- ----- ----- ------- ------- -------- -------- ---- ... ------ ------ ------ ------ ------ ------ ------ ------


# Cross Match with SDSS

In [20]:
import pyvo

# Connect to MAST TAP service
sdss_tap = pyvo.dal.TAPService("https://datalab.noirlab.edu/tap")

# List available tables (optional)
for name in sdss_tap.tables.keys():
    if "sdss" in name.lower():
        print(name)

allwise.x1p5__source__sdss_dr12__specobj
allwise.x1p5__source__sdss_dr16__specobj
allwise.x1p5__source__sdss_dr17__specobj
catwise2020.x1p5__main__sdss_dr17__specobj
decaps_dr1.x1p5__object__sdss_dr17__specobj
decaps_dr2.x1p5__object__sdss_dr17__specobj
delve_dr1.x1p5__objects__sdss_dr17__specobj
delve_dr2.x1p5__objects__sdss_dr17__specobj
des_dr1.x1p5__main__sdss_dr17__specobj
des_dr2.x1p5__main__sdss_dr17__specobj
desi_dr1.x1p5__zpix__sdss_dr17__specobj
desi_edr.x1p5__zpix__sdss_dr17__specobj
euclid_q1.x1p5__object__sdss_dr17__specobj
gaia_dr1.x1p5__gaia_source__sdss_dr16__specobj
gaia_dr3.x1p5__gaia_source__sdss_dr12__specobj
gaia_dr3.x1p5__gaia_source__sdss_dr16__specobj
gaia_dr3.x1p5__gaia_source__sdss_dr17__specobj
gnirs_dqs.x1p5__spec_measurements__sdss_dr17__specobj
gogreen_dr1.x1p5__photo__sdss_dr17__specobj
gogreen_dr1.x1p5__redshift__sdss_dr17__specobj
gogreen_dr2.x1p5__photo__sdss_dr17__specobj
gogreen_dr2.x1p5__redshift__sdss_dr17__specobj
ivoa_sdss_dr9.exposure
ivoa_sdss_

In [31]:
import math

# Inputs: ra, dec in degrees; radius_arcsec in arcsec
ra0   = float(ra)
dec0  = float(dec)
rdeg  = float(5000) / 3600.0

# RA padding (deg); protect cos(dec)=0 at poles
rapad = rdeg / max(math.cos(math.radians(dec0)), 1e-6)

# RA wrap-around condition
if (ra0 - rapad) < 0.0 or (ra0 + rapad) >= 360.0:
    ra_cond = f"(s.ra >= {(ra0 - rapad) % 360:.8f} OR s.ra <= {(ra0 + rapad) % 360:.8f})"
else:
    ra_cond = f"s.ra BETWEEN {ra0 - rapad:.8f} AND {ra0 + rapad:.8f}"

# Dec clamp
dec_min = max(-90.0, dec0 - rdeg)
dec_max = min( 90.0, dec0 + rdeg)

adql_query = f"""
SELECT TOP 500
  s.specobjid, s.ra, s.dec, s.class, s.z
FROM sdss_dr17.specobj AS s
WHERE
  {ra_cond}
  AND s.dec BETWEEN {dec_min:.8f} AND {dec_max:.8f}
  AND DEGREES(ACOS(
        SIN(RADIANS({dec0:.10f})) * SIN(RADIANS(s.dec)) +
        COS(RADIANS({dec0:.10f})) * COS(RADIANS(s.dec)) * COS(RADIANS(s.ra - {ra0:.10f}))
      )) <= {rdeg:.10f}
"""

job = sdss_tap.run_sync(adql_query)
tbl = job.to_table()
tbl.pprint(max_width=120)


specobjid  ra dec class  z 
          deg deg          
--------- --- --- ----- ---


In [32]:
# Count spectra within 1.5 degrees (~5400")
rdeg = 1.5
ra0  = float(ra)
dec0 = float(dec)

adql = f"""
SELECT COUNT(*) AS n_spec
FROM sdss_dr17.specobj AS s
WHERE
  DEGREES(ACOS(
    SIN(RADIANS({dec0})) * SIN(RADIANS(s.dec)) +
    COS(RADIANS({dec0})) * COS(RADIANS(s.dec)) * COS(RADIANS(s.ra - {ra0}))
  )) <= {rdeg}
"""
sdss_tap.run_sync(adql).to_table().pprint()


n_spec
------
     0


# Pan-STARRSS1 DR1 (VizieR: II/349/ps1)

In [34]:
from pyvo.dal import TAPService
import numpy as np
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
import astropy.units as u
import math

# your transient position (degrees)
ra0  = float(ra)
dec0 = float(dec)

# search radius for host association
r_arcsec = 60.0           # e.g., 60″; adjust as you like
rdeg = r_arcsec / 3600.0

viz = TAPService("https://tapvizier.u-strasbg.fr/TAPVizieR/tap")

# Small helper: great-circle cone in ADQL with trig (server-agnostic)
cone_where = f"""
DEGREES(ACOS(
  SIN(RADIANS({dec0})) * SIN(RADIANS(DEJ2000)) +
  COS(RADIANS({dec0})) * COS(RADIANS(DEJ2000)) * COS(RADIANS(RAJ2000 - {ra0}))
)) <= {rdeg}
"""

# Query Pan-STARRS1 mean photometry DR1
q_ps1 = f"""
SELECT TOP 1000
  objID, RAJ2000, DEJ2000, gmag, rmag, imag, zmag, ymag
FROM "II/349/ps1"
WHERE {cone_where}
"""

ps1 = viz.run_sync(q_ps1).to_table()

# Compute separations and pick nearest
if len(ps1) > 0:
    c_tns  = SkyCoord(ra0*u.deg, dec0*u.deg)
    c_ps1  = SkyCoord(ps1["RAJ2000"]*u.deg, ps1["DEJ2000"]*u.deg)
    sep    = c_tns.separation(c_ps1).to(u.arcsec)
    ps1["sep_arcsec"] = sep.value
    ps1.sort("sep_arcsec")
    host_ps1 = ps1[0]  # nearest source
    print("Nearest PS1 source (candidate host):")
    print(host_ps1)
else:
    print("No PS1 sources found within", r_arcsec, "arcsec.")


KeyboardInterrupt: 